In [ ]:
pip install pandas sqlalchemy mysql-connector-python

In [3]:
#Librerías necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import os
from bs4 import BeautifulSoup
from docx import Document
from sqlalchemy import create_engine

In [ ]:
############################ WEBSCRAPPING FALABELLA ############################

In [ ]:
# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# Ruta del driver
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Cambia esta ruta si es necesario
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Lista de productos (enlaces y modelos)
F_productos = [
    # Xiaomi
    ("https://www.falabella.com.co/falabella-co/product/127913189/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Negro/127913190", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127913775/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Amarillo/127913776", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127916553/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5G-512Gb-12Ram-200Mp-Negro/127916554", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/128104744/Celular-Xiaomi-Redmi-Note-13-PRO-PLUS-5G-512GB-12GB-RAM-Negro/128104745", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/130055074/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5g-512gb-12ram-PURPURA/130055075", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp"),
    ("https://www.falabella.com.co/falabella-co/product/127687136/CELULAR-XIAOMI-REDMI-13C-256GB-8GB-RAM-NEGRO/127687137", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128261646/Celular-Xiaomi-Redmi-13C-256GB-8RAM-Negro/128261647", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128128076/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Negro/128128077", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128807587/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Purpura/128807589", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/prod13530339/Celular-Xiaomi-Redmi-Note-13-Pro-256GB-8GB-Camara-Posterior-200MP+8MP+2MP-Camara-Frontal-16MP-Pantalla-6.67-Pulgadas/72853341", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128128195/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Forest-Green/128128196", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram"),
    ("https://www.falabella.com.co/falabella-co/product/128931528/Celular-Xiaomi-Redmi-Note-13-8GB-256GB-Negro/128931529", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/127891485/Celular-Xiaomi-Redmi-Note-13-256GB-8GB-RAM-Negro/127891486", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB"),
    
    # Samsung
    ("https://www.falabella.com.co/falabella-co/product/129586393/CELULAR-SAMSUNG-GALAXY-A55-5G-256GB-8-RAM-AZUL-NAVY/129586394", "Samsung", "SAMSUNG GALAXY A55"),
    ("https://www.falabella.com.co/falabella-co/product/129554425/Celular-Samsung-Galaxy-A55-5G-256Gb-8Ram-Azul-Oscuro/129554428", "Samsung", "SAMSUNG GALAXY A55"),
    ("https://www.falabella.com.co/falabella-co/product/prod13530496/Celular-Samsung-A15-256GB-8GB-RAM-Camara-Posterior-50-MP-Camara-Frontal-12MP-Pantalla-6.5-Pulgadas-+-MediaTek-MT6789V-CD-/72854633", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/132236410/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL-NEGRO/132236411", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/128004219/CELULAR-SAMSUNG-GALAXY-A15-256GB-8-RAM-NEGRO-(Blue-Black)/128004220", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/132236494/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL/132236495", "Samsung", "Samsung A15 256GB"),
    ("https://www.falabella.com.co/falabella-co/product/128906319/Celular-Samsung-Galaxy-A25-5G-256GB-8RAM+-Cargador/128906321", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM"),
    ("https://www.falabella.com.co/falabella-co/product/131931272/Celular-SAMSUNG-GALAXY-A25-256GB-+-8-RAM-AZUL-CLARO/131931273", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM"),
    ("https://www.falabella.com.co/falabella-co/product/72899217/Celular-Samsung-Galaxy-A35-5G-256GB-8GB-RAM-camara-posterior-50-MP-camara-frontal-13MP-pantalla-6.6-Pulgadas-+-Exynos-1380/72899217", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/133744591/Celular-Samsung-Galaxy-A35-256Gb-5g-Lila/133744592", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/128668464/Celular-Samsung-Galaxy-A05S-128Gb-Negro/128668465", "Samsung", "Samsung Galaxy A05S 128Gb"),
    ("https://www.falabella.com.co/falabella-co/product/128035979/Samsung-S24-Ultra-256GB-12GB-Violet/128035980", "Samsung", "Samsung S24 Ultra 256GB 12GB"),
    ("https://www.falabella.com.co/falabella-co/product/128853893/Celular-Samsung-Galaxy-S24-Ultra-De-256GB12GB-RAM-Gris-5G/128853894", "Samsung", "Samsung S24 Ultra 256GB 12GB"),
    ("https://www.falabella.com.co/falabella-co/product/129574090/Samsung-Galaxy-S24-Ultra-512GB-12GB-Ram-5G-Violeta/129574091", "Samsung", "Samsung Galaxy S24 Ultra 512GB 12GB"),

    #Apple
    ("https://www.falabella.com.co/falabella-co/product/prod12000048/iPhone-13-128-GB-5G-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic/19604508", "Apple", "iPhone 13 128 GB 5G 4GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod13430667/iPhone-15-de-128GB-5G-6GB-RAM-Pantalla-6.1-pulgadas-Chip-A16-Bionic-Carga-Tipo-C-Dynamic-Island-Camara-48MP/72752105", "Apple", "iPhone 15 128GB 5G 6GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod13070047/iPhone-14-128GB-5G-6GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic-/60999138", "Apple", "iPhone 14 128GB 5G 6GB RAM"),
    ("https://www.falabella.com.co/falabella-co/product/prod10900027/iPhone-11-128-GB-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A13-Bionic-Camara-12-MP/9722720", "Apple", "iPhone 11 128 GB 4GB RAM"),
]

# Lista para almacenar los datos
F_data = []

# Iterar sobre cada producto
for url, marca, modelo in F_productos:
    print(f"Procesando: {modelo} ({marca})")

    # Cargar la página del producto
    driver.get(url)

    # Esperar a que se cargue el contenedor de comentarios
    try:
        wait = WebDriverWait(driver, 25)  # Aumentar si es necesario para conexiones lentas
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='user-reviews-container']")))

        # Cargar todos los comentarios
        comentarios = []
        while True:
            try:
                ver_mas_btn = driver.find_element(By.ID, "more-comments-button")
                driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)
                ver_mas_btn.click()

                # Esperar dinámicamente hasta que se carguen nuevos comentarios
                WebDriverWait(driver, 25).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")) > len(comentarios)
                )

                # Actualizar el número de comentarios cargados
                comentarios = driver.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                print(f"Comentarios cargados hasta ahora: {len(comentarios)}")  # Log de progreso

            except Exception:
                print("No hay más comentarios para cargar.")
                break

        # Extraer los comentarios y calificaciones
        F_reviews = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='review']")
        for review in reviews:
            try:
                # Extraer el comentario
                comentario_element = review.find_element(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                comentario = comentario_element.text.strip()

                # Buscar las estrellas dentro del contenedor actual
                estrellas_llenas = review.find_elements(By.CSS_SELECTOR, "span._star-full_10acl_10")
                calificacion = len(estrellas_llenas)  # Contar las estrellas llenas

                print(f"Comentario: {comentario[:30]}... | Calificación: {calificacion}")  # Log para depuración

                # Guardar en la lista
                F_data.append({"comentario": comentario, "calificacion": calificacion, "marca": marca, "modelo": modelo})

            except Exception as e:
                print(f"Error procesando una review: {e}")

    except Exception as e:
        print(f"Error procesando {modelo}: {e}")

# Crear un DataFrame con los resultados
F_df = pd.DataFrame(F_data)

# Exportar a CSV
F_df.to_csv("comentarios_productos_falabella.csv", index=False, encoding="utf-8")
print(f"Se procesaron {len(data)} comentarios en total. Archivo guardado como 'comentarios_productos.csv'.")

# Cerrar el navegador
driver.quit()

Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [ ]:
F_df.groupby(['marca', 'modelo']).size()

In [21]:
############################ WEBSCRAPPING MERCADOLIBRE ############################

In [22]:
# Ruta de la carpeta que contiene los documentos
folder_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Webscrapping\Mercadolibre\HTMLs Mercadolibre"

# Lista para almacenar los datos de todos los archivos
all_data = []

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):  # Verificar si el archivo es .docx
        file_path = os.path.join(folder_path, filename)

        # Leer el contenido del documento .docx
        doc = Document(file_path)
        html_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

        # Analizar el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extraer los comentarios
        comments = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
        extracted_comments = [comment.get_text(strip=True) for comment in comments]

        # Extraer las calificaciones
        ratings = []
        rating_containers = soup.find_all('div', class_='ui-review-capability-comments__comment__rating')
        for container in rating_containers:
            stars = len(container.find_all('use', href='#poly_star_fill'))
            ratings.append(stars)

        # Asegurar que comentarios y calificaciones tengan el mismo tamaño
        if len(extracted_comments) > len(ratings):
            ratings.extend([None] * (len(extracted_comments) - len(ratings)))
        elif len(ratings) > len(extracted_comments):
            extracted_comments.extend([None] * (len(ratings) - len(extracted_comments)))

        # Agregar los datos al conjunto general
        for comment, rating in zip(extracted_comments, ratings):
            all_data.append({'Filename': filename, 'Comment': comment, 'Rating': rating})

# Crear el DataFrame con el nombre "mercadolibre_reviews"
mercadolibre_reviews = pd.DataFrame(all_data)

In [23]:
#DATA CLEANING
# Eliminar ".docx" si existe
mercadolibre_reviews['Filename'] = mercadolibre_reviews['Filename'].str.replace('.docx', '', regex=False)

# Extraer el modelo (todo lo que está antes del precio)
mercadolibre_reviews['Model'] = mercadolibre_reviews['Filename'].str.extract(r'^(.*)\s\$\d+')

# Eliminar la columna "Filename"
mercadolibre_reviews = mercadolibre_reviews.drop(columns=['Filename'])

# Reorganizar las columnas en el orden especificado
mercadolibre_reviews = mercadolibre_reviews[['Model', 'Comment', 'Rating']]

In [24]:
# Diccionario de mapeo del modelo al código
model_to_code = {
    "IPhone 13": "FI1",
    "IPhone 15": "FI2",
    "IPhone 14": "FI3",
    "IPhone 11": "FI4",
    "Apple iPhone 11 (128 GB) - Negro": "FI5",
    "Apple iPhone 13 (128 GB) - Azul": "FI6",
    "Apple iPhone 14 (128 GB) - Medianoche": "FI7",
    "Apple iPhone 15 (128 GB) - Negro": "FI8",
    "Samsung Galaxy A55": "FS1",
    "Samsung Galaxy A15": "FS3",
    "Samsung Galaxy A25": "FS7",
    "Samsung Galaxy A35": "FS9",
    "Samsung Galaxy A05": "FS11",
    "Samsung S24 Ultra": "FS12",
    "Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM": "FS15",
    "Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM": "FS16",
    "Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM": "FS17",
    "Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM": "FS18",
    "Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM": "FS19",
    "Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM": "FS20",
    "Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM": "FS21",
    "Xiaomi Poco X6 Pro": "FX1",
    "Xiaomi Redmi Note 13 Pro Plus": "FX3",
    "Xiaomi Redmi 13C": "FX6",
    "Xiaomi Redmi Note 13 Pro": "FX8",
    "Xiaomi Redmi Note 13": "FX12",
    "Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB negro 12 GB RAM": "FX14",
    "Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM": "FX15",
    "Xiaomi Redmi A3 Dual SIM 64 GB midnight black 3 GB RAM": "FX16",
    "Xiaomi Redmi Note 13 4G Dual SIM 256 GB negro 8 GB RAM": "FX17",
    "Xiaomi Redmi Note 13 Pro 4G Dual SIM 256 GB púrpura 8 GB RAM": "FX18",
    "Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB Negro 12 GB RAM": "FX19"
}

# Agregar la columna "Codigo" basada en el mapeo
mercadolibre_reviews['Codigo'] = mercadolibre_reviews['Model'].map(model_to_code)

# Guardar el DataFrame en un archivo CSV
mercadolibre_reviews.to_csv('all_comments_and_ratings.csv', index=False)

# Verificar el resultado
mercadolibre_reviews

,Model,Comment,Rating,Codigo
0,Apple iPhone 11 (128 GB) - Negro,"Es un dispositivo magnifico, se adapta a la ne...",5,FI5
1,Apple iPhone 11 (128 GB) - Negro,"Excelente precio para un equipo tan completo, ...",5,FI5
2,Apple iPhone 11 (128 GB) - Negro,No sabía que llegaba sin cargador.,4,FI5
3,Apple iPhone 11 (128 GB) - Negro,Ha sido excelente este producto. El celular ha...,5,FI5
4,Apple iPhone 11 (128 GB) - Negro,"Buen producto, totalmente nuevo.",5,FI5
...,...,...,...,...
2494,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,Es un buen teléfono gama media. Y muy bonito. ...,5,FX19
2495,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,,5,FX19
2496,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,Lo compre en el rush de un producto nuevo del ...,5,FX19
2497,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,"Perfecto , su cámara es muy buena , el diseño ...",5,FX19


In [25]:
#Conteo de comentarios extraidos
# Agrupar por "Filename" y contar los comentarios únicos
grouped_comments = mercadolibre_reviews.groupby('Model')['Comment'].count()

print(grouped_comments)

Model
Apple iPhone 11 (128 GB) - Negro                                         83
Apple iPhone 13 (128 GB) - Azul                                         180
Apple iPhone 14 (128 GB) - Medianoche                                    50
Apple iPhone 15 (128 GB) - Negro                                        130
Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM                           90
Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM              173
Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM                            71
Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM                       106
Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM              210
Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM      210
Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM    111
Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB negro 12 GB RAM              210
Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM                210
Xiaomi

In [26]:
###################################### EXPORTAR DATAFRAMES A MYSQL #########################################

In [27]:
# Configura los detalles de la conexión
user = 'root'  # Cambia según tu configuración
password = 'Elefantes1507'  # Cambia según tu configuración
host = 'localhost'  # O 127.0.0.1
port = '3306'  # Puerto predeterminado de MySQL
database = 'sentimental_analysis_db'  # Nombre de tu base de datos

# Crear el motor de conexión
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")

In [28]:
# Guardar los DataFrames en MySQL
# Asegúrate de que mercadolibre_reviews y F_df ya están definidos como DataFrames
mercadolibre_reviews.to_sql('mercadolibre_reviews', engine, if_exists='replace', index=False)
#F_df.to_sql('f_reviews', engine, if_exists='replace', index=False)

print("DataFrames cargados exitosamente en MySQL.")

DataFrames cargados exitosamente en MySQL.
